##1.Batch Normalization

* covariate shift : 학습하는 과정에서 이전 layer의 파라미터 변화로 인해 현재 layer의 분포가 바뀌는 현상
* 학습과정에서는 분포가 비슷할수록 좋다.
* 이는 건축현상에서의 휘어짐(Buckling)현상과 비슷하다. (뉴럴넷의 붕괴?를 방지하려면 BN을 해야함)
* 이를 줄이기 위해서는 whitening : 입력을 평균 0, 분산 1로 바꾸는 것(정규분포화), 역전파와 무관하게 하면 안된다
* whitening과 BN의 차이는 평균과 분산을 조절하는 과정이 신경망 안에 포함되어 있어 학습과정에서 평균과 분산을 조절

####Batch Norm 방법
* 학습 전체 과정에서 하는 것이 제일 좋지만 (역전파에 용이?) 파라미터의 업데이트가 Mini batch 단위로 진행되므로 Mini batch 단위로 batch Norm
* 평균과 분산을 구한 후 정규화->정규화 이후에 scale(alpha)와 shift(beta)연산을 진행
* 학습에서는 mini batch마다 scale와 shift를 구하고 그 값을 저장, 테스트 과정에서 그 평균을 사용
* FC layer로만 이루어진 신경망에 적용하면 학습효과가 더 좋음

![대체 텍스트](https://www.learnopencv.com/wp-content/uploads/2018/07/val_loss.png)




In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


def BN_forward(x, gamma, beta, eps):
    N, D = x.shape
    mu = 1./N * np.sum(x, axis = 0)
    xmu = x - mu
    var = 1./N * np.sum(xmu**2, axis = 0)
    sqrtvar = np.sqrt(var + eps)
    ivar = 1./sqrtvar
    xhat = xmu * ivar
    out = (gamma * xhat) + beta
    values = (xhat,gamma,xmu,ivar,sqrtvar,var,eps)
    return out, values


![대체 텍스트](https://shuuki4.files.wordpress.com/2016/01/bn1.png)

In [0]:

# Backward 메서드입니다.
def BN_backward(dout, cache):
    xhat,gamma,xmu,ivar,sqrtvar,var,eps = cache
    N,D = dout.shape
    
    dbeta = np.sum(dout, axis=0)
    dgammax = dout #dr/dx= dr/dl * dl/dx ?
    dgamma = np.sum(dgammax*xhat, axis=0)
    

    dxhat = dgammax*gamma
    # (dl/dvar) 를 계산합니다.
    divar = np.sum(dxhat*xmu, axis=0)
    dsqrtvar = -1. /(sqrtvar**2) * divar
    dvar = 0.5 * 1. /np.sqrt(var+eps) * dsqrtvar
    
    
    dxmu1 = dxhat * ivar
    dsq = 1. /N * np.ones((N,D)) * dvar
    dxmu2 = 2 * xmu * dsq
    
    
    dx1 = dxmu1+dxmu2
    
    
    dmu = -1 * np.sum(dxmu1+dxmu2, axis=0)
    dx2 = 1. /N * np.ones((N,D)) * dmu
    
    .
    dx = dx1+dx2
    
    return dx, dgamma, dbeta



[batch normalization 역전파 설명 링크](https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html)
![대체 텍스트](https://kratzert.github.io/images/bn_backpass/BNcircuit.png)


In [0]:

def ReLU(x):
    return np.maximum(0,x)

def histogram(x, title):
    plt.hist(x)
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Freqency')
    plt.savefig('plot.png')
    elice_utils.send_image("plot.png")


np.random.seed(50)
x = np.random.rand(50,50)

gamma, beta, epsilon = 1, 0, 10e-7
histogram(x, 'Origin')


out, cache = BN_forward(x,gamma,beta,epsilon)
dx, dgamma, dbeta = BN_backward(out,cache)
histogram(dx, 'Backward')

out = ReLU(gamma)
histogram(out,'Forward')



   


## 2.transfer learning

* 잘 훈련된 모델을 사용하여 유사한 문제를 해결하는 방법
* 학습을 위해서는 많은 데이터가 필요
* 모델의 일부분을 변경했을 때 다시 전체를 학습하는 과정 필요 but 고사양 컴퓨터도 많은 시간 소요

###동작 과정
* 빅 데이터셋을 사용해 초기 신경망 모델 학습
* 학습된 신경망 모델에 사용자 데이터셋을 사용하여 학습

###전이학습 학습 방법
![대체 텍스트](https://miro.medium.com/max/600/0*PvR62SNNcJZf3uGc.jpg)

* 트레이닝 데이터의 크기와 트레이닝 데이터의 유사도에 따라 다르다.
* 유사도도 놓고 데이터도 제일 많으면 좋음.
* 그러나 대부분 데이터 유사도가 높아도 데이터셋의 크기가 작음
*

###지식 증류
* Ensemble(앙상블)
 * 동일한 구조에서 Initialization을 다르게 여러번 학습하거나 다른 구조의 NN을 학습하고 나오는 결과물을 합치는 과정, 여러 모델을 돌리기 위한 컴퓨터 자원이 필요
* Distilling the Ensemble Knowledge
 * 신경망의 지식 증류는 불필요한 파라미터가 사용되는 앙상블 모델에서 Eneralization 성능을 향상 시킬 수 있는 지식들을 분리하는 것
 * 이미지 분류에서 최종 출력은 Softmax를 사용
 * 앙상블 모델의 softmax 결과를 새로운 NN이 잘 전달 받는다면

* 학습 과정은 Softmax 출력이 Label과 최대한 비슷해 지도록 Softmax cross entropy Loss를 최소화 하는 방식으로 학습하는데 이런 신경망에는 많은 정보가 담겨져 있음

* Softer Softmax

![대체 텍스트](https://bloglunit.files.wordpress.com/2018/03/20180322-research-seminar-google-slides-2018-03-22-17-24-15.png?w=226&h=149)

 파라미터 T를 추가하여 T가 높을수록 기존보다 조금 더 soft한 확률분포를 얻을 수 있도록 함
 (마치 불순물 추출하는 과정에서 온도를 조절하는 것, 이를 조절함으로써 fine-tune을 더 잘 할 수 있게 해줄 수 있음)